# Goal:
- Pull NIFC Firedata in realtime via API call
- Preprocess and clean data
- Display fire perimeter on a map

### Add Fire Perimeters

In [13]:
import requests
import os
from datetime import datetime
import json

# Current Global GeoService API 
current_url = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/WFIGS_Interagency_Perimeters_Current/FeatureServer/0/query"

# Historical API
historical_url = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/InterAgencyFirePerimeterHistory_All_Years_View/FeatureServer/0/query"

# Global year to date API
year_to_date_url = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/WFIGS_Interagency_Perimeters_YearToDate/FeatureServer/0/query"

# California 2024 -2026
cali_url = "https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/CA_Perimeters_NIFC_FIRIS_public_view/FeatureServer/0/query"

In [29]:
# test params
params = {
    "where": "1=1", # SQL-style filter
    "outFields": "*", # specify which fields to return
    "returnGeometry": "true",
    "f": "geojson", # format
    "outSR": 4326, # spatial reference WGS84 latitude/longitude (EPSG:4326)
    "resultRecordCount": 2000 # page size
}


response = requests.get(current_url, params=params, timeout=30)

# check for http success
if response.status_code == 200:
    geojson_data = response.json() # turn into json
    # print(json.dumps(geojson_data, indent=4)) # pretty print results
else:
    print(f"Failed to retrieve GeoJSON data. Status code: {response.status_code}")

### Add perscribed fires
**Data source:**
https://watchduty.maps.arcgis.com/home/item.html?id=1784011a31104a45a584b26dbd23adfe  
- Updates regularly

**Use Constraints and Permissions:**
Commercial or for-profit use of this service is strictly prohibited without prior written consent from Sherwood Forestry Service, dba Watch Duty. If you have additional relevant prescribed burn data you would like to contribute, please submit it for review to: prescribedfire@watchduty.org.


In [36]:
perscribed_url = "https://services5.arcgis.com/VNhSlpl1umSknM3q/arcgis/rest/services/Watch_Duty_Prescribed_Fires/FeatureServer/0/query"

response = requests.get(perscribed_url, params=params, timeout=30)

# check for http success
if response.status_code == 200:
    perscribed_geojson_data = response.json() # turn into json
    # print(json.dumps(perscribed_geojson_data, indent=4)) # pretty print results
else:
    print(f"Failed to retrieve GeoJSON data. Status code: {response.status_code}")


### Add layers to Map

In [37]:
import folium

m = folium.Map([40.0, -96.0], zoom_start=4, tiles="cartodb positron")

folium.GeoJson(geojson_data, name="fire perimeters").add_to(m)
folium.GeoJson(perscribed_geojson_data, name="perscribed fires").add_to(m)
folium.LayerControl().add_to(m)

In [38]:
m